## 데이터셋 설명
- 본 데이터셋은 텍스트 정보와 직무에 대한 메타 정보로 구성되어 있습니다.
- 데이터셋에는 길이 제한이 없는 잡 포스팅과, 해당 포스팅이 진짜인지, 혹은 가짜인지의 여부가 포함되어 있습니다.
- 자연어(Natrual Language) 데이터, 그 중에서도 영어 데이터를 전처리하여 텍스트 데이터를 딥러닝에 적용하는 과정을 통해, 데이터 전처리와 특징 추출의 과정을 배워보시길 바랍니다.

## 자연어 처리 (Natural Language Processing, NLP)
- 자연어 데이터를 머신러닝에 사용하기 위해서는 데이터를 머신러닝에 사용할 수 있도록 전처리하는 과정이 필요합니다.
- 일반적으로 머신러닝에 사용되는 데이터가 어떤 형태이고, 자연어가 이와 어떻게 다른지 생각해봅시다.
    - 데이터의 크기: 대부분의 머신러닝 모델들은 고정된 크기의 입력 데이터를 받습니다. 그러나, 자연어는 문장에 따라 길이가 상이합니다. 때문에 자연어를 머신러닝 모델에 투입하려면 데이터를 고정된 크기로 변환해줘야 합니다.
    - 데이터의 형태: 머신러닝 모델들은 실수 데이터를 입력 받습니다. 그러나, 자연어 데이터는 문자형(char, string)으로 되어있습니다.
- 이러한 문제들로 자연어 처리에서는 전처리 방식이 매우 중요합니다.

In [ ]:
# 라이브러리 임포트
import os
import random
from PIL import Image

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from tqdm import tqdm

from sklearn.metrics import accuracy_score

import torchvision
from torchvision.models import VGG16_Weights
from torchvision.transforms import v2

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# 하이퍼파라미터
args = {
    "train_path" : "/content/gdrive/MyDrive/p1/train.csv",      # train 데이터 경로
    "test_path" : "/content/gdrive/MyDrive/p1/test.csv",       # test 데이터 경로
    "submit_path" : "/content/gdrive/MyDrive/p1/sample_submission.csv",     # submit 파일 경로
    "batch_size" : 64,
    "epochs" : 1,
    "lr" : 2e-5,
    "seed_val" : 42         # 절대 수정하지 마세요.
}

In [ ]:
# 랜덤시드 고정하기
seed = args["seed_val"]
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available() :
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

# 디바이스 선택
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
%pip install torchsummary ## model을 요약하기 위해 torchsummary 설치
from torchsummary import summary as summary_## 모델 정보를 확인하기 위해 torchsummary 함수 import

## 모델의 형태를 출력하기 위한 함수
def summary_model(model, input_shape=(3,32,32)):
    model = model.to(device)
    summary_(model, input_shape) ## (model, (input shape))

# **1. 데이터셋**

## **(1) 데이터 불러오기 및 전처리**

In [ ]:
# 훈련 및 테스트 데이터 로드
train = pd.read_csv(args['train_path'])
test = pd.read_csv(args['test_path'])

# (1-1) 훈련 데이터에서 특징(x_train)과 라벨(y_train) 분리
x_train = train.iloc[:,1:]# # 첫 번째 열을 제외한 모든 열 (특징)
y_train = train['label']# 'label' 열 (라벨)
x_test = test # 테스트 데이터

# (1-2) numpy 배열로 변환 후, torch Tensor로 변환
x_train = torch.from_numpy(x_train.values).float()
y_train = torch.from_numpy(y_train.values).long()
x_test = torch.from_numpy(x_test.values).float()

# (1-3) 데이터를 (N, H, W, C) 형태로 재구성 (reshape 함수 이용)
x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)

# (1-4) 데이터를 (N, C, H, W) 형태로 변환 (permute 함수 이용)
x_train = x_train.permute(0,3,1,2)
x_test = x_test.permute(0,3,1,2)

# (1-5) 데이터를 (N, 3, 28, 28) 형태로 변환 (expand 함수 이용)
x_train = x_train.expand(-1,3,-1,-1)
x_test = x_test.expand(-1,3,-1,-1)

In [ ]:
x_train.shape

torch.Size([27455, 3, 28, 28])

In [ ]:
x_test.shape

torch.Size([7172, 3, 28, 28])

## **(2) 데이터셋과 데이터로더**

In [ ]:
# 훈련 및 테스트 데이터셋 로드
train_dataset = TensorDataset(x_train, y_train)
test_dataset = TensorDataset(x_test)

# 데이터로더 정의
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=args["batch_size"], shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

# **2. 모델**
사전학습된 VGG16을 사용하여 프로젝트를 수행하세요.

## **(1) VGG16 특징 추출기를 불러오기**

In [ ]:
# (3-1) VGG16 모델 로드 및 특징 추출 부분 사용
# 가중치는 'VGG16_Weights.IMAGENET1K_V1' 사용하세요.
model = torchvision.models.vgg16(weights=VGG16_Weights.IMAGENET1K_V1)

# (3-2) 모델에 글로벌 평균 풀링 계층 추가
model.global_avg_pool2d = nn.AdaptiveAvgPool2d(output_size=(1,1))

# 모델 요약 정보 출력
summary_model(model)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:05<00:00, 94.2MB/s]


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
              ReLU-2           [-1, 64, 32, 32]               0
            Conv2d-3           [-1, 64, 32, 32]          36,928
              ReLU-4           [-1, 64, 32, 32]               0
         MaxPool2d-5           [-1, 64, 16, 16]               0
            Conv2d-6          [-1, 128, 16, 16]          73,856
              ReLU-7          [-1, 128, 16, 16]               0
            Conv2d-8          [-1, 128, 16, 16]         147,584
              ReLU-9          [-1, 128, 16, 16]               0
        MaxPool2d-10            [-1, 128, 8, 8]               0
           Conv2d-11            [-1, 256, 8, 8]         295,168
             ReLU-12            [-1, 256, 8, 8]               0
           Conv2d-13            [-1, 256, 8, 8]         590,080
             ReLU-14            [-1, 25

## **(2) VGG16 특징 추출기를 동결시키기**
1. 계산 비용 절감:
    - 모델의 일부 파라미터를 동결하면 그 부분은 학습되지 않기 때문에 계산량이 줄어듭니다.

    - 이는 학습 속도를 빠르게 하고, GPU 메모리 사용량을 줄이는 데 도움이 됩니다.

2. 오버피팅 방지:
    - 작은 데이터셋을 사용할 때 모델이 쉽게 오버피팅될 수 있습니다.
    
    - 이미 학습된 파라미터를 동결하면 모델이 새로운 데이터에 맞춰 과도하게 학습되는 것을 방지할 수 있습니다.


3. 전이 학습(Transfer Learning):
    - 사전 학습된 모델(예: ImageNet 데이터셋으로 학습된 모델)의 초반 레이어는 일반적인 특징(예: 에지, 텍스처 등)을 잘 잡아냅니다.

    - 이러한 특징을 이용하면 새로운 데이터셋에서도 좋은 성능을 낼 수 있습니다.

    - 모델의 후반 레이어만 재학습하여 새로운 데이터셋에 맞출 수 있습니다.

4. 더 빠른 수렴:
    - 동결된 파라미터는 변화하지 않으므로 모델이 더 빠르게 수렴할 수 있습니다.
    
    - 이는 전체 학습 시간을 단축시키는 데 도움이 됩니다.

In [ ]:
# 모델의 앞부분 파라미터 고정 (동결)
for para in model.features[:-8].parameters():
    para.requires_grad = False

# 파라미터 동결 후 모델 요약 정보 출력
summary_model(model)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
              ReLU-2           [-1, 64, 32, 32]               0
            Conv2d-3           [-1, 64, 32, 32]          36,928
              ReLU-4           [-1, 64, 32, 32]               0
         MaxPool2d-5           [-1, 64, 16, 16]               0
            Conv2d-6          [-1, 128, 16, 16]          73,856
              ReLU-7          [-1, 128, 16, 16]               0
            Conv2d-8          [-1, 128, 16, 16]         147,584
              ReLU-9          [-1, 128, 16, 16]               0
        MaxPool2d-10            [-1, 128, 8, 8]               0
           Conv2d-11            [-1, 256, 8, 8]         295,168
             ReLU-12            [-1, 256, 8, 8]               0
           Conv2d-13            [-1, 256, 8, 8]         590,080
             ReLU-14            [-1, 25

## **(3) VGG16 분류기를 만들기**

In [1]:
# (4-1) 새로운 분류기 정의
classifier = nn.Sequential(  ## 이거 바꾸는 듯
    # 25개의 클래스로 분류
    nn.Linear(512 * 1 * 1, 4096),  # VGG16의 마지막 conv layer 출력이 512 채널이고, AdaptiveAvgPool2d로 1x1로 줄였으므로
    nn.BatchNorm1d(4096)
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(4096, 1024),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(1024, 25)  # 최종 출력을 25개 클래스로 설정
)

# (4-2) 모델에 새로운 분류기 추가
model.classifier = classifier
model.avgpool = nn.AdaptiveAvgPool2d((1, 1))

# 모델을 장치로 이동
model = model.to(device)

# 최종 모델 요약 정보 출력
summary_model(model)

NameError: name 'nn' is not defined

# **3. 학습**

In [ ]:
# VGG16에서 사용한 transform을 이용
transform = VGG16_Weights.DEFAULT.transforms(antialias=False)

# resize 크기는 32가 되도록 설정
transform.resize_size=[32]

# transform 확인하기
print(transform)

ImageClassification(
    crop_size=[224]
    resize_size=[32]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)


In [ ]:
import torch.optim as optim
def train(train_dataloader, model, device, args):
    """
    주어진 데이터로 모델을 학습시키는 함수입니다.

    Args:
        train_dataloader (DataLoader): 학습 데이터를 제공하는 DataLoader 객체
        valid_dataloader (DataLoader): 검증 데이터를 제공하는 DataLoader 객체
        model (torch.nn.Module): 학습할 모델
        device (torch.device): 사용할 디바이스 (CPU 또는 GPU)
        args (dict): 학습 관련 인자들을 포함한 딕셔너리

    Returns:
        None
    """

    # (5-1) Adam 옵티마이저와 교차 엔트로피 손실 함수 정의
    optimizer = optim.Adam(model.parameters(), lr=args["lr"])
    loss_fn = nn.CrossEntropyLoss()

    model.zero_grad()  # 모델의 그래디언트 초기화

    for epoch in range(args["epochs"]):
        model.train()       # 모델을 훈련 모드로 설정

        total_loss = 0      # 전체 손실 초기화
        total_accuracy = 0  # 전체 정확도 초기화

        print(f'Epoch {epoch + 1}/{args["epochs"]}')

        for image, label in tqdm(train_dataloader):
            image = transform(image).to(device)
            label = label.to(device)

            # (5-2) 모델을 사용하여 예측 수행
            pred = model(image)

            # (5-3) 손실 계산 및 누적
            loss = loss_fn(pred, label)

            # (5-4) 역전파를 통해 기울기 계산
            loss.backward()

            # (5-5) 파라미터 업데이트
            optimizer.step()

            # (5-6) 모델의 그래디언트 초기화
            optimizer.zero_grad()

            # loss 계산
            total_loss += loss.item()

            # accaracy 계산
            label = label.cpu()
            pred = pred.argmax(dim = 1).cpu()
            accuracy = accuracy_score(label, pred)
            total_accuracy += accuracy

        # 평균 손실과 정확도 계산
        avg_loss = total_loss / len(train_dataloader)
        avg_accuracy = total_accuracy / len(train_dataloader)

        # 모델 체크포인트 저장
        os.makedirs("results", exist_ok=True)
        torch.save({
            'epoch': epoch,
            'model': model,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss.item,
            }, f'./results/model_state_dict_epoch_{epoch+1}.pth')

        # 현재 에포크의 손실과 정확도 출력
        print(f'CheckPoint : model_state_dict_epoch_{epoch+1}.pth')
        print(f'train_loss : {avg_loss}, train_acc : {avg_accuracy}\n')


if __name__ == "__main__":
    train(train_dataloader, model, device, args)

Epoch 1/1


 56%|█████▋    | 242/429 [01:33<01:11,  2.60it/s]


KeyboardInterrupt: 

# **4. 평가**

In [ ]:
def test(test_dataloader, model, device):
    """
    모델의 테스트를 수행하는 함수입니다.

    Args:
        test_dataloader (DataLoader): 테스트 데이터를 제공하는 DataLoader 객체
        model (torch.nn.Module): 평가할 모델
        device (torch.device): 사용할 디바이스 (CPU 또는 GPU)

    Returns:
        preds (list): 각 입력 예시에 대한 모델의 예측 결과 리스트
    """
    model.eval()    # 모델을 평가 모드로 설정
    preds = []      # 예측 결과를 저장할 리스트

    # 각 배치에 대해 예측 수행
    for image in tqdm(test_dataloader):
        image = transform(image[0]).to(device)

        # 기울기 계산을 비활성화하여 예측 수행
        with torch.no_grad():
            # (6-1) 모델에 입력을 전달하여 예측 수행
            pred = model(image)

            # (6-2) argmax를 이용하여 예측 결과에서 가장 높은 값의 인덱스를 선택
            pred = pred.argmax(dim = 1)

            # (6-3) 예측 결과를 CPU로 이동
            pred = pred.cpu()

            # (6-4) 예측을 numpy 배열로 변환
            pred = pred.numpy()

            # (6-5) 예측 결과를 리스트에 추가
            preds.append(pred)

    return preds


if __name__ == "__main__":
    # 예측값을 얻기 위해 test 함수 호출
    preds = test(test_dataloader, model, device)

In [ ]:
submit = pd.read_csv(args["submit_path"])
submit["label"] = preds
submit.to_csv("submission_p1.csv", index = False)

In [ ]:
submit